In [83]:
import pandas as pd

def array4DataFrame(array):
    # 配列に対して、扱いやすいデータフレーム形式（ヘッダー付き）に変換する
    # 行番号を付与して各行の一番目に追加
    for i, row in enumerate(array, start=1):
        row[0] = f"{row[0]}_{i}"

    # Columnの一行目を取得し、ヘッダーとして再利用する
    header_list = ["name"]
    for i in range(len(array)):
        header_list.append(array[i][0])
    df = pd.DataFrame(array, columns=header_list)
    
    return df


data = [["knowledge", 0, 0.1, 0, 0.4, 0, 0.3, 0.5],
            ["knowledge", 0.1, 0, 0.4, 0, 0.6, 0, 0],
            ["knowledge", 0, 0.4, 0, 0.2, 0, 0, 0.7],
            ["knowledge", 0.4, 0, 0.2, 0, 0.1, 0, 0],
            ["knowledge", 0, 0.6, 0, 0.1, 0, 0.4, 0.2],
            ["knowledge", 0.3, 0, 0, 0, 0.4, 0, 0.6],
            ["knowledge", 0.5, 0, 0.7, 0, 0.2, 0.6, 0]]

df = array4DataFrame(data)

# 拡散する先をソートして返します。
def next_diffusion_pair(df,stimulus):#sapnetモジュール内で使用
    diffusion_list = []
    column = df.iloc[:, stimulus]
    for i in range(len(column)):
        distance_value = column[i]
        if distance_value > 0.0:
            diffusion_list.append([distance_value,stimulus,i+1])
            # 配列を一番前の少数を基準にソート
            sorted_diffusion_list = sorted(diffusion_list, key=lambda x: x[0])
            #print(sorted_diffusion_list)
            # 今回の拡散が行われるリスト
            pair_list = [sublist[1:] for sublist in sorted_diffusion_list]
            #print(pair_list)
            # 次のパス一覧を返却
            path_list = [sublist[1:] for sublist in pair_list]
            next_list = [[sublist[0] for sublist in path_list]]
    return pair_list

def already_pair_remove(pair_list,already_list):#sapnetモジュール内で使用
    return_list = []
    for pair_temp in pair_list:
        if pair_temp not in already_list:
            return_list.append(pair_temp)
            already_list.append(pair_temp)
            reversed_par = pair_temp[::-1]
            already_list.append(reversed_par)
        else:
            None
    return return_list,already_list

# ノードのパス数
def path_num_calc(df,stimulus):
    row_number = stimulus - 1  # 行番号を調整
    selected_row = df.iloc[row_number]
    count_positive = 0  # 0以上の値のカウントを初期化
    stimulus = float(stimulus)  # stimulusを数値に変換
    for value in selected_row[1:]:  # 最初の列（name列）を除いて値を処理
        if value > 0 and value != stimulus:
            count_positive += 1  # 0以上の値があればカウントを増やす
    return count_positive

# パスの重み
def path_weight_calc(df, stimulus_number,receive_number):
    row_number = stimulus_number-1# 例として2を指定
    selected_row = df.iloc[row_number]
    column_number = receive_number
    if stimulus_number != receive_number:
        selected_value = selected_row[column_number]
        return selected_value



In [48]:
print(df)

          name  knowledge_1  knowledge_2  knowledge_3  knowledge_4   
0  knowledge_1          0.0          0.1          0.0          0.4  \
1  knowledge_2          0.1          0.0          0.4          0.0   
2  knowledge_3          0.0          0.4          0.0          0.2   
3  knowledge_4          0.4          0.0          0.2          0.0   
4  knowledge_5          0.0          0.6          0.0          0.1   
5  knowledge_6          0.3          0.0          0.0          0.0   
6  knowledge_7          0.5          0.0          0.7          0.0   

   knowledge_5  knowledge_6  knowledge_7  
0          0.0          0.3          0.5  
1          0.6          0.0          0.0  
2          0.0          0.0          0.7  
3          0.1          0.0          0.0  
4          0.0          0.4          0.2  
5          0.4          0.0          0.6  
6          0.2          0.6          0.0  


In [84]:
def sapnet(df,stimulus):
    already_list=[]
    temp_list = [stimulus]
    #print("temp",temp_list)
    while len(temp_list)!=0:
        #print(i)
        now_list = temp_list
        temp_list = []
        for item in range(len(now_list)):
            #print("item",now_list[item])
            pair_list = next_diffusion_pair(df,now_list[item])
            #print("pair",pair_list)
            check_pair_list,already_list = already_pair_remove(pair_list,already_list)
            #print("al",already_list)
            #print("check",check_pair_list)
            path_list = [sublist[1:] for sublist in check_pair_list]
            next_list = [sublist[0] for sublist in path_list]
            #print("next",next_list)
            temp_list.extend(next_list)
            #print("check",check_pair_list)
            for active_pair in check_pair_list:
                print(active_pair)
                offer_num,receive_num = active_pair[0],active_pair[1]
                #print("刺激したノード",offer_num)
                #print("刺激されたノード",receive_num)
                #print(path_num_calc(df,offer_num))#パス数
                N = path_num_calc(df,offer_num)
                #print(path_weight_calc(df,offer_num,receive_num))#パスの重み
                w = path_weight_calc(df,offer_num,receive_num)
                
                
                
    #print("temp",temp_list)

In [85]:
stimulus = 1
sapnet(df,stimulus)

[1, 2]
4
0.1
[1, 6]
4
0.3
[1, 4]
4
0.4
[1, 7]
4
0.5
[2, 3]
3
0.4
[2, 5]
3
0.6
[6, 5]
3
0.4
[6, 7]
3
0.6
[4, 5]
3
0.1
[4, 3]
3
0.2
[7, 5]
4
0.2
[7, 3]
4
0.7


In [38]:
num = path_num_calc(df,stimulus)
print(num)

4
